# **JUMBO JUMJAI**

An Optimization Hackathon Project

In [355]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

## 1. **Data Preprocessing**

In [356]:
!pip install pulp

In [357]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

In [358]:
data = pd.read_csv('/content/WIFI_modified.csv')
data.head()

,Timestamp,AP Name,Status,Total user,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,AP Type,Uninstallation Cost
0,3/1/2025 0:00,AP7426.ac5b.af38,"ma 662d,10h,48m",7,96.51%,37.454012,-52,NaN,Moderate,17,27.890109,4000,1000,1,500
1,3/1/2025 0:00,CMU_AGR_AGRI_AP0182,"up 14d,19h,24m",6,98.21%,95.071431,-77,NaN,Mild,15,30.697188,5000,1500,2,700
2,3/1/2025 0:00,CMU_AGR_AGRI_AP0183,"up 70d,13h,4m",3,98.85%,73.199394,-71,High,Severe,7,33.415107,4000,1000,1,500
3,3/1/2025 0:00,CMU_AGR_AGRI_AP0184,"up 4d,8h,9m",8,98.84%,59.865848,-56,NaN,Moderate,20,78.562128,5000,1500,2,700
4,3/1/2025 0:00,CMU_AGR_AGRI_AP0185,"up 48d,7h,59m",2,96.78%,15.601864,-50,NaN,Mild,5,13.745678,5000,1500,2,700


In [359]:
# Split "AP Name" into components
dfsplit = data["AP Name"].str.split("_", expand=True)
dfsplit.columns = ["AP1", "AP2", "AP3", "APID"][:dfsplit.shape[1]]

# Combine "AP1", "AP2", and "AP3" into a single column
dfsplit["AP Name"] = dfsplit[["AP1", "AP2", "AP3"]].fillna('').agg('_'.join, axis=1).str.strip('_')

# Drop "AP1", "AP2", and "AP3"
dfsplit = dfsplit.drop(columns=["AP1", "AP2", "AP3"])

# Combine with the remaining columns
data = pd.concat([
    data[["Timestamp", "Status", "Total user", "Availability",
        "Bandwidth Utilization", "Signal Strength (RSSI)",
        "Packet Loss & Latency", "Interference Level", "Device Count",
        "Session Duration", "Installation Cost", "Maintenance Cost","AP Type", "Uninstallation Cost"]],
    dfsplit
], axis=1)

# Reorder columns to move 'APID' and 'AP_Combined' to the front
data = data[["APID", "AP Name"] + [col for col in data.columns if col not in ["APID", "AP Name"]]]

In [360]:
# Extract the last word from "AP Name" to "APID"
data['Last_Word_AP_Name'] = data['AP Name'].str.split('_').str[-1]
data['APID'] = data['Last_Word_AP_Name'] + '_' + data['APID']
data = data.drop(columns=['Last_Word_AP_Name'])

# Rename APID where None to AP0100
data.loc[data["APID"].isnull(), "APID"] = "AP0000"

# Convert Maintenance Cost to 0 where Status is up and running
data.loc[data['Status'].str.contains('up', case=False), 'Maintenance Cost'] = 0

# Split Timestamp to Date and Time
data[['Date', 'Time']] = data['Timestamp'].str.split(' ', expand=True)
data = data.drop('Timestamp', axis=1)
data['Date'] = pd.to_datetime(data['Date'])
data['Time'] = pd.to_datetime(data['Time'], errors='coerce').dt.time

# Convert 'Availability' to numeric, handling errors
data['Availability'] = pd.to_numeric(data['Availability'].str.replace('%', ''), errors='coerce')

# Update 'Total user' and 'Device Count' based on 'Status'
data.loc[data['Status'].str.contains('ma|down', case=False, na=False), ['Total user', 'Device Count']] = 0
data.head()

<ipython-input-360-c4ce366a547c>:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Time'] = pd.to_datetime(data['Time'], errors='coerce').dt.time


,APID,AP Name,Status,Total user,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,AP Type,Uninstallation Cost,Date,Time
0,AP0000,AP7426.ac5b.af38,"ma 662d,10h,48m",0,96.51,37.454012,-52,NaN,Moderate,0,27.890109,4000,1000,1,500,2025-03-01,00:00:00
1,AGRI_AP0182,CMU_AGR_AGRI,"up 14d,19h,24m",6,98.21,95.071431,-77,NaN,Mild,15,30.697188,5000,0,2,700,2025-03-01,00:00:00
2,AGRI_AP0183,CMU_AGR_AGRI,"up 70d,13h,4m",3,98.85,73.199394,-71,High,Severe,7,33.415107,4000,0,1,500,2025-03-01,00:00:00
3,AGRI_AP0184,CMU_AGR_AGRI,"up 4d,8h,9m",8,98.84,59.865848,-56,NaN,Moderate,20,78.562128,5000,0,2,700,2025-03-01,00:00:00
4,AGRI_AP0185,CMU_AGR_AGRI,"up 48d,7h,59m",2,96.78,15.601864,-50,NaN,Mild,5,13.745678,5000,0,2,700,2025-03-01,00:00:00


In [361]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172368 entries, 0 to 172367
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   APID                    172368 non-null  object        
 1   AP Name                 172368 non-null  object        
 2   Status                  172368 non-null  object        
 3   Total user              172368 non-null  int64         
 4   Availability            172368 non-null  float64       
 5   Bandwidth Utilization   172368 non-null  float64       
 6   Signal Strength (RSSI)  172368 non-null  int64         
 7   Packet Loss & Latency   129720 non-null  object        
 8   Interference Level      128736 non-null  object        
 9   Device Count            172368 non-null  int64         
 10  Session Duration        172368 non-null  float64       
 11  Installation Cost       172368 non-null  int64         
 12  Maintenance Cost        172368

**Define uninstallation costs based on Installation Cost**

In [362]:
# uninstallation_costs = {
#     3000: 450,  # For Installation Cost 3000
#     4000: 600,  # For Installation Cost 4000
#     5000: 750   # For Installation Cost 5000
# }

# # Add the 'Uninstallation Cost' column
# data['Uninstallation Cost'] = data['Installation Cost'].map(uninstallation_costs)
# data.head()

# 2. 24 Hours Average Data
Convert 172368 rows (24 hrs) -> 7182 rows (24 hrs avg)

In [363]:
# Convert Object by using Mode
def most_frequent(series):
    return series.mode()[0] if not series.mode().empty else None

data = data.groupby(["APID", "Date"]).agg({
            "AP Name": "first",
            "Status": "first",
            "Total user": "mean",
            "Availability": "mean",
            "Bandwidth Utilization": "mean",
            "Signal Strength (RSSI)": "mean",
            "Packet Loss & Latency": most_frequent,
            "Interference Level": most_frequent,
            "Device Count": "mean",
            "Session Duration": "mean",
            "Installation Cost": "first",
            "Maintenance Cost": "first",
            "AP Type": "first",
            "Uninstallation Cost": "first"
        }).reset_index()

data.head()

,APID,Date,AP Name,Status,Total user,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,AP Type,Uninstallation Cost
0,40YD_AP8359,2025-03-01,CMU_ENG_40YD,"up 84d,8h,44m",5.083333,99.698750,48.332658,-55.458333,None,Mild,9.875000,110.198275,3000,0,0,300
1,40YD_AP8360,2025-03-01,CMU_ENG_40YD,"up 11d,19h,34m",2.250000,99.172083,19.929650,-70.916667,High,None,4.083333,29.023893,4000,0,1,500
2,40YD_AP8361,2025-03-01,CMU_ENG_40YD,"up 50d,19h,34m",3.791667,99.788750,61.890893,-34.291667,High,Severe,6.250000,49.187809,5000,0,2,700
3,40YD_AP8533,2025-03-01,TOT_ENG_40YD,"up 84d,8h,54m",3.250000,97.605000,96.926939,-75.416667,High,Moderate,5.708333,101.641828,4000,0,1,500
4,40YD_AP8534,2025-03-01,TOT_ENG_40YD,"ma 147d,14h,45m",0.000000,99.772083,83.879881,-34.833333,None,Severe,0.000000,27.252927,4000,1000,1,500


Rounding Total Average User to Integer

In [364]:
data = data.rename(columns={"Total user": "Total Average User"})
data["Total Average User"] = data["Total Average User"].round(0).astype(int)
data.head()

,APID,Date,AP Name,Status,Total Average User,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,AP Type,Uninstallation Cost
0,40YD_AP8359,2025-03-01,CMU_ENG_40YD,"up 84d,8h,44m",5,99.698750,48.332658,-55.458333,None,Mild,9.875000,110.198275,3000,0,0,300
1,40YD_AP8360,2025-03-01,CMU_ENG_40YD,"up 11d,19h,34m",2,99.172083,19.929650,-70.916667,High,None,4.083333,29.023893,4000,0,1,500
2,40YD_AP8361,2025-03-01,CMU_ENG_40YD,"up 50d,19h,34m",4,99.788750,61.890893,-34.291667,High,Severe,6.250000,49.187809,5000,0,2,700
3,40YD_AP8533,2025-03-01,TOT_ENG_40YD,"up 84d,8h,54m",3,97.605000,96.926939,-75.416667,High,Moderate,5.708333,101.641828,4000,0,1,500
4,40YD_AP8534,2025-03-01,TOT_ENG_40YD,"ma 147d,14h,45m",0,99.772083,83.879881,-34.833333,None,Severe,0.000000,27.252927,4000,1000,1,500


# 3. Assign Location Name to Each APID

In [365]:
print("Total Unique APID =", data["APID"].nunique())
print("Total Unique AP Name =", data['AP Name'].nunique())

Total Unique APID = 7181
Total Unique AP Name = 166


Take a look at the Location.csv

In [366]:
location = pd.read_csv('/content/Location.csv')
location.tail()

,AP2_AP3,Location1
143,ENG_GENT2,หอพักชาย2
144,ENG_LADY11,หอพักหญิง11
145,ENG_LADY12,หอพักหญิง12
146,ENG_RIMKLONG,แฟลตริมคลอง
147,ENG_SCMC,ที่พักรถม่วงหน้าหอสาม


In [367]:
location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   AP2_AP3    148 non-null    object
 1   Location1  148 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


In [368]:
location['AP2_AP3'].unique()

array(['AGR_AGRI', 'AGR_ARTMU', 'AGR_ASSET', 'AGR_BA', 'AGR_BUS',
       'AGR_CLEANWT', 'AGR_CULT', 'AGR_EDU', 'AGR_FIN', 'AGR_GRAD',
       'AGR_LI', 'AGR_OD', 'AGR_OOP', 'AGR_PTRI', 'AGR_RIHES', 'AGR_SRI',
       'AGR_STRI', 'AGR_VET', 'COM_AKD', 'COM_ASSET', 'COM_CAMT',
       'COM_CELEBRATE', 'COM_CMUHEART', 'COM_CNOC', 'COM_COFFAK',
       'COM_COOP', 'COM_ECON', 'COM_FCMU', 'COM_HISTORY', 'COM_HM',
       'COM_ITSC', 'COM_ITSCC', 'COM_LAW', 'COM_LIBR', 'COM_MASS',
       'COM_OD', 'COM_OOP', 'COM_POL', 'COM_POST', 'COM_REG', 'COM_SAVE',
       'COM_SCB', 'COM_SCI', 'COM_SLAK', 'COM_SLDM', 'COM_SOC',
       'COM_SOCF', 'COM_SPP', 'COM_SWC', 'COM_VCMU', 'ENG_40YD',
       'ENG_AMS', 'ENG_ARC', 'ENG_ARCHER', 'ENG_CAMT', 'ENG_CONTAINER',
       'ENG_DMST', 'ENG_ENG', 'ENG_FAI', 'ENG_FLAT1', 'ENG_FLAT2',
       'ENG_FLAT3', 'ENG_FLAT4', 'ENG_FLAT7', 'ENG_FLAT8', 'ENG_FOODCEN',
       'ENG_GENT3', 'ENG_GENT4', 'ENG_GENT5', 'ENG_GENT6', 'ENG_GENT7',
       'ENG_GH', 'ENG_GYM', 'ENG_LADY

In [369]:
# Remove "CMU_" or "TOT_" from "AP Name"
data["AP Name"] = data["AP Name"].str.replace("CMU_", "", regex=False)
data["AP Name"] = data["AP Name"].str.replace("TOT_", "", regex=False)
print("Total Unique Location =", location['AP2_AP3'].nunique())
print("Total Unique AP Name =", data['AP Name'].nunique())

Total Unique Location = 148
Total Unique AP Name = 149


In [370]:
ap_names_data = set(data["AP Name"].unique())
ap_names_location = set(location["AP2_AP3"].unique())

# Find elements in data but not in location
diff1 = ap_names_data - ap_names_location
print("AP Names in 'data' but not in 'location':", diff1)

# Find elements in location but not in data
diff2 = ap_names_location - ap_names_data
print("AP Names in 'location' but not in 'data':", diff2)

AP Names in 'data' but not in 'location': {'AP7426.ac5b.af38'}
AP Names in 'location' but not in 'data': set()


**Merge the data and location DataFrames based on "AP Name" and "AP2_AP3"**

In [371]:
data = pd.merge(data, location, left_on='AP Name', right_on='AP2_AP3', how='left')
data = data.drop(columns=['AP2_AP3'])
data.head()

,APID,Date,AP Name,Status,Total Average User,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,AP Type,Uninstallation Cost,Location1
0,40YD_AP8359,2025-03-01,ENG_40YD,"up 84d,8h,44m",5,99.698750,48.332658,-55.458333,None,Mild,9.875000,110.198275,3000,0,0,300,หอสี่สิบปี
1,40YD_AP8360,2025-03-01,ENG_40YD,"up 11d,19h,34m",2,99.172083,19.929650,-70.916667,High,None,4.083333,29.023893,4000,0,1,500,หอสี่สิบปี
2,40YD_AP8361,2025-03-01,ENG_40YD,"up 50d,19h,34m",4,99.788750,61.890893,-34.291667,High,Severe,6.250000,49.187809,5000,0,2,700,หอสี่สิบปี
3,40YD_AP8533,2025-03-01,ENG_40YD,"up 84d,8h,54m",3,97.605000,96.926939,-75.416667,High,Moderate,5.708333,101.641828,4000,0,1,500,หอสี่สิบปี
4,40YD_AP8534,2025-03-01,ENG_40YD,"ma 147d,14h,45m",0,99.772083,83.879881,-34.833333,None,Severe,0.000000,27.252927,4000,1000,1,500,หอสี่สิบปี


In [372]:
data.tail()

,APID,Date,AP Name,Status,Total Average User,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,AP Type,Uninstallation Cost,Location1
7176,VET_AP7823,2025-03-01,MAE_VET,"up 1d,15h,54m",8,95.212083,34.714214,-42.666667,Low,None,11.875000,22.247423,3000,0,0,300,แม่เหียะ ศูนย์สุขภาพสัตว์เลี้ยง
7177,VET_AP7824,2025-03-01,MAE_VET,"ma 86d,10h,44m",0,92.808333,72.860357,-73.375000,High,Moderate,0.000000,101.999980,5000,1500,2,700,แม่เหียะ ศูนย์สุขภาพสัตว์เลี้ยง
7178,VET_AP7825,2025-03-01,MAE_VET,"up 1d,15h,54m",3,95.166667,65.684364,-32.750000,Low,Mild,5.750000,33.601841,3000,0,0,300,แม่เหียะ ศูนย์สุขภาพสัตว์เลี้ยง
7179,WAT_AP6409,2025-03-01,ENG_WAT,"up 28d,11h,24m",4,99.258333,46.164563,-45.833333,Medium,None,6.833333,51.556984,3000,0,0,300,วัดฝายหิน
7180,WAT_AP6667,2025-03-01,ENG_WAT,"up 4d,19h,34m",2,98.793750,7.461262,-48.666667,Medium,Mild,3.625000,97.792896,4000,0,1,500,วัดฝายหิน


In [373]:
# Split 'Status' column into 'Status Type' and 'Status Time'
data[['Status Type', 'Status Time']] = data['Status'].str.split(' ', n=1, expand=True)

# Drop the old 'Status' column
data.drop(columns=['Status'], inplace=True)

data.head()


,APID,Date,AP Name,Total Average User,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,AP Type,Uninstallation Cost,Location1,Status Type,Status Time
0,40YD_AP8359,2025-03-01,ENG_40YD,5,99.698750,48.332658,-55.458333,None,Mild,9.875000,110.198275,3000,0,0,300,หอสี่สิบปี,up,"84d,8h,44m"
1,40YD_AP8360,2025-03-01,ENG_40YD,2,99.172083,19.929650,-70.916667,High,None,4.083333,29.023893,4000,0,1,500,หอสี่สิบปี,up,"11d,19h,34m"
2,40YD_AP8361,2025-03-01,ENG_40YD,4,99.788750,61.890893,-34.291667,High,Severe,6.250000,49.187809,5000,0,2,700,หอสี่สิบปี,up,"50d,19h,34m"
3,40YD_AP8533,2025-03-01,ENG_40YD,3,97.605000,96.926939,-75.416667,High,Moderate,5.708333,101.641828,4000,0,1,500,หอสี่สิบปี,up,"84d,8h,54m"
4,40YD_AP8534,2025-03-01,ENG_40YD,0,99.772083,83.879881,-34.833333,None,Severe,0.000000,27.252927,4000,1000,1,500,หอสี่สิบปี,ma,"147d,14h,45m"


In [374]:
# Drop rows with NaN values in the 'Location1' column
data = data.dropna(subset=['Location1'])

In [375]:
data.to_csv('Wi-Fi_data.csv', index=False)

'''from google.colab import files
files.download('Wi-Fi_data.csv')'''

"from google.colab import files\nfiles.download('Wi-Fi_data.csv')"

In [376]:
import pandas as pd
import pulp

# โหลดข้อมูล
file_path = "/content/Wi-Fi_data.csv"
df = pd.read_csv(file_path)
# จัดกลุ่มข้อมูลตามสถานที่และประเภท AP
location_groups = df.groupby(["Location1", "AP Type"])
location_status = location_groups["Status Type"].value_counts(normalize=True).unstack(fill_value=0)
location_counts = location_groups["APID"].count()

# แปลง Status Type และ Status Time
df["Status Type"] = df["Status Type"].astype(str)
df["Status Time"] = df["Status Time"].astype(str)

# ค่าต้นทุนและความสามารถของแต่ละประเภท AP
ap_types = {
    0: {"install": 3000, "repair": 500, "capacity": 50, "remove": 300},
    1: {"install": 4000, "repair": 1000, "capacity": 70, "remove": 500},
    2: {"install": 5000, "repair": 1500, "capacity": 100, "remove": 700},
}


# ตั้งค่า Optimization Model
model = pulp.LpProblem("Minimize_WiFi_Upgrade_Cost", pulp.LpMinimize)

# สร้างตัวแปรติดตั้ง/ซ่อมแซม/ถอน ตามแต่ละ AP Type
install_vars = {}
repair_vars = {}
remove_vars = {}
for (loc, ap_type), count in location_counts.items():
    install_vars[(loc, ap_type)] = pulp.LpVariable(f"Install_{loc}_Type{ap_type}", lowBound=0, cat="Integer")
    repair_vars[(loc, ap_type)] = pulp.LpVariable(f"Repair_{loc}_Type{ap_type}", lowBound=0, cat="Integer")
    remove_vars[(loc, ap_type)] = pulp.LpVariable(f"Remove_{loc}_Type{ap_type}", lowBound=0, cat="Integer")

# ฟังก์ชันวัตถุประสงค์: ลดค่าใช้จ่ายทั้งหมด
total_cost = (
    pulp.lpSum(install_vars[(loc, ap)] * ap_types[ap]["install"] for (loc, ap) in install_vars) +
    pulp.lpSum(repair_vars[(loc, ap)] * ap_types[ap]["repair"] for (loc, ap) in repair_vars) +
    pulp.lpSum(remove_vars[(loc, ap)] * ap_types[ap]["remove"] for (loc, ap) in remove_vars)
)
model += total_cost

In [377]:
# Constraints:
# 1. แต่ละสถานที่ต้องมี AP ใช้งาน 80% ขึ้นไป
for (loc, ap_type), count in location_counts.items():
    total_ap = count
    up_ap_needed = int(0.8 * total_ap)
    current_up_ap = int(location_status.loc[(loc, ap_type), "up"] * total_ap) if "up" in location_status.columns else 0
    model += (current_up_ap + repair_vars[(loc, ap_type)] + install_vars[(loc, ap_type)]) >= up_ap_needed


In [378]:
# 2. ถอนการติดตั้ง AP หาก ma > 365 วัน
for idx, row in df.iterrows():
    if row["Status Type"] == "ma" and "d" in row["Status Time"]:
        days = int(row["Status Time"].split("d")[0])
        if days > 365:
            model += remove_vars[(row["Location1"], row["AP Type"])] >= 1

In [379]:
# 3. งบประมาณต้องไม่เกิน 40 ล้านบาท
model += total_cost <= 40_000_000

In [380]:
# 4. ต้องรองรับผู้ใช้ได้ 38,000 คนขึ้นไป
total_users = pulp.lpSum(
    (install_vars[(loc, ap)] + repair_vars[(loc, ap)] + location_counts.get((loc, ap), 0) - remove_vars[(loc, ap)]) * ap_types[ap]["capacity"]
    for (loc, ap) in install_vars.keys()
)
model += total_users >= 38_000


In [381]:
# 5. ต้องมีจุด AP มากกว่า 0
for (loc, ap_type) in location_counts.keys():
    model += install_vars[(loc, ap_type)] + repair_vars[(loc, ap_type)] + location_counts[(loc, ap_type)] - remove_vars[(loc, ap_type)] >= 1


In [382]:
# Solve Model
model.solve()

1

In [383]:
# สรุปผลลัพธ์
results = []
for (loc, ap_type) in location_counts.keys():
    results.append({
        "Location": loc,
        "AP Type": ap_type,
        "Install AP": int(install_vars[(loc, ap_type)].varValue),
        "Repair AP": int(repair_vars[(loc, ap_type)].varValue),
        "Remove AP": int(remove_vars[(loc, ap_type)].varValue),
        "Total Cost": int(
            install_vars[(loc, ap_type)].varValue * ap_types[ap_type]["install"] +
            repair_vars[(loc, ap_type)].varValue * ap_types[ap_type]["repair"] +
            remove_vars[(loc, ap_type)].varValue * ap_types[ap_type]["remove"]
        )
    })


In [384]:
# สร้าง DataFrame แสดงผลลัพธ์
df_results = pd.DataFrame(results)

In [385]:
# บันทึกผลลัพธ์
df_results.to_csv("output.csv", index=False)

In [386]:
df_results.head()

,Location,AP Type,Install AP,Repair AP,Remove AP,Total Cost
0,TLIC,0,0,0,1,300
1,TLIC,1,0,0,0,0
2,TLIC,2,0,0,1,700
3,AIS Playgroud,0,0,0,0,0
4,AIS Playgroud,1,0,0,0,0


In [387]:
# prompt: sum of total cost

# Calculate the sum of the 'Total Cost' column
total_cost_sum = df_results['Total Cost'].sum()

print(f"The sum of total cost is: {total_cost_sum}")


The sum of total cost is: 324000


In [388]:
    # คำนวณค่าใช้จ่ายรวมของแต่ละสถานที่
    total_cost_per_location = df_results.groupby("Location")["Total Cost"].sum().reset_index()
    total_cost_per_location.columns = ["Location", "Total Cost"]

    # แสดงผลลัพธ์
    print("ค่าใช้จ่ายรวมของแต่ละสถานที่:")
    print(total_cost_per_location)

ค่าใช้จ่ายรวมของแต่ละสถานที่:
                 Location  Total Cost
0                    TLIC        1000
1           AIS Playgroud           0
2                     AMS           0
3      Ang kaeo Dormitory           0
4                    CAMT       12500
..                    ...         ...
139  โรงปรับปรุงคุณภาพน้ำ           0
140        โรงพยาบาลสัตว์           0
141        โรงอาหาร สังคม           0
142            ไทยพาณิชย์           0
143              ไปรษณีย์           0

[144 rows x 2 columns]


In [389]:
# prompt:  i want to see all rows in total_cost_per_location

import pandas as pd
# Display all rows of total_cost_per_location
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print("ค่าใช้จ่ายรวมของแต่ละสถานที่:")
total_cost_per_location


ค่าใช้จ่ายรวมของแต่ละสถานที่:


,Location,Total Cost
0,TLIC,1000
1,AIS Playgroud,0
2,AMS,0
3,Ang kaeo Dormitory,0
4,CAMT,12500
...,...,...
139,โรงปรับปรุงคุณภาพน้ำ,0
140,โรงพยาบาลสัตว์,0
141,โรงอาหาร สังคม,0
142,ไทยพาณิชย์,0


In [390]:
# prompt: แสดงจำนวนซ้ำของค่าใน location

# Display the value counts for the 'Location1' column
location_counts = data['Location1'].value_counts()
location_counts


,count
Location1,
คณะแพทยศาสตร์,1290
คณะวิศวะฯ,484
คณะวิทยาสาสตร์ คอม,464
หอพักนักศึกษาพยาบาล 1,287
คณะศึกษาศาสตร์,261
...,...
สนามยิงธนู,1
แม่เหียะ สนามฟุตบอล,1
ไปรษณีย์,1
